In [1]:
import numpy as np
import re
import pandas as pd

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
text = pd.read_excel('ruscorpora_content.xlsx')
text = np.array(text['Full context'].to_list()).flatten()
text = np.array2string(text)
text = text.replace('\ufeff', '')  # убираем первый невидимый символ
text = re.sub(r'[^А-я ]', '', text)  # заменяем все символы кроме кириллицы на пустые символы
text

'Эффективность общего как государствакорпорации в виде инфраструктурных подкорпораций оказывающих конкурентный мировой сервис как самой себе так и частному бизнесу все дороги порты морские аэро космическая связь актуальность создания группировки низкоорбитальных спутников для организации широкополосного интернета всюду и космической мобильной связи транспорт и т д  Производство товаров массового спроса и розничная торговля сегодня не только демонстрируют высокую динамику они достигли той стадии зрелости когда от решения точечных или инфраструктурных задач требуется переход к задачам оптимизации бизнеса  Кроме того управление государственными предприятиями уже само по себе может выполнять функцию регулирования данной отрасли особенно если эти предприятия носят инфраструктурный характер  Электронный бизнес придуман для удовлетворения потребностей определенного типа предприятий в определенной экономической и инфраструктурной среде  Стоит отметить что объединяться с другими фармацевтически

In [3]:
# парсим текст, как последовательность символов
num_characters = 34  # 33 буквы + пробел
tokenizer = Tokenizer(num_words=num_characters, char_level=True)  # токенизируем на уровне символов
tokenizer.fit_on_texts([text])  # формируем токены на основе частотности в нашем тексте
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'и': 3, 'е': 4, 'н': 5, 'р': 6, 'т': 7, 'а': 8, 'с': 9, 'в': 10, 'к': 11, 'у': 12, 'л': 13, 'п': 14, 'м': 15, 'д': 16, 'ы': 17, 'я': 18, 'г': 19, 'з': 20, 'ь': 21, 'х': 22, 'б': 23, 'й': 24, 'ф': 25, 'ч': 26, 'ж': 27, 'ц': 28, 'ю': 29, 'э': 30, 'ш': 31, 'щ': 32, 'ъ': 33}


In [4]:
inp_chars = 3
data = tokenizer.texts_to_matrix(text)  # преобразуем исходный текст в массив OHE

# мы предсказываем 4 символ после 3, поэтому, чтобы предсказать последний символ в data на надо закончить
# формировать X за 3 символа до конца
n = data.shape[0] - inp_chars  


X = np.array([data[i:i + inp_chars, :] for i in range(n)])
Y = np.array([data[i + inp_chars, :] for i in range(n)])# предсказание следующего символа

print(data.shape)

(64245, 34)


In [5]:
X[0], Y[0]

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0.]]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [6]:
model = Sequential()
model.add(Input((inp_chars,
                 num_characters)))  # при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
model.add(SimpleRNN(200, activation='tanh', return_sequences=True))
model.add(SimpleRNN(100, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3, 200)            47000     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               30100     
                                                                 
 dense (Dense)               (None, 34)                3434      
                                                                 
Total params: 80534 (314.59 KB)
Trainable params: 80534 (314.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.fit(X, Y, batch_size=50, epochs=100, validation_split=0.1)

Epoch 1/100
1157/1157 [==============================] - 3s 2ms/step - loss: 2.3440 - accuracy: 0.3255 - val_loss: 2.1854 - val_accuracy: 0.3538
Epoch 2/100
1157/1157 [==============================] - 3s 2ms/step - loss: 2.0372 - accuracy: 0.4015 - val_loss: 2.0342 - val_accuracy: 0.3970
Epoch 3/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.9060 - accuracy: 0.4378 - val_loss: 1.9460 - val_accuracy: 0.4195
Epoch 4/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.8186 - accuracy: 0.4574 - val_loss: 1.8926 - val_accuracy: 0.4328
Epoch 5/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.7608 - accuracy: 0.4738 - val_loss: 1.8452 - val_accuracy: 0.4503
Epoch 6/100
1157/1157 [==============================] - 3s 3ms/step - loss: 1.7172 - accuracy: 0.4825 - val_loss: 1.8252 - val_accuracy: 0.4577
Epoch 7/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.6833 - accuracy: 0.4906 - val_loss: 1.8113 - val_ac

1157/1157 [==============================] - 3s 2ms/step - loss: 1.3908 - accuracy: 0.5465 - val_loss: 1.6801 - val_accuracy: 0.4981
Epoch 58/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.3888 - accuracy: 0.5460 - val_loss: 1.6960 - val_accuracy: 0.4918
Epoch 59/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.3883 - accuracy: 0.5471 - val_loss: 1.6845 - val_accuracy: 0.4953
Epoch 60/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.3864 - accuracy: 0.5457 - val_loss: 1.6870 - val_accuracy: 0.4903
Epoch 61/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.3834 - accuracy: 0.5461 - val_loss: 1.6981 - val_accuracy: 0.4864
Epoch 62/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.3821 - accuracy: 0.5480 - val_loss: 1.6973 - val_accuracy: 0.4940
Epoch 63/100
1157/1157 [==============================] - 3s 2ms/step - loss: 1.3809 - accuracy: 0.5465 - val_loss: 1.6858 - val_accuracy

In [9]:
def buildPhrase(inp_str, str_len=25):
    for i in range(str_len):
        x = []
        for j in range(i, i + inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j]))  # преобразуем символы в One-Hot-encoding

        x = np.array(x)
        inp = x.reshape(1, inp_chars, num_characters)

        pred = model.predict(inp)  # предсказываем OHE четвертого символа
        d = tokenizer.index_word[pred.argmax(axis=1)[0]]  # получаем ответ в символьном представлении

        inp_str += d  # дописываем строку

    return inp_str


res = buildPhrase("пре")
print(res)

1/1 [==============================] - 0s 8ms/step
предприятия в развития в раз
